In [109]:
import os
import shutil
import sys
import torch
import yaml
import numpy as np
from datetime import datetime

In [113]:
config = yaml.load(open("config.yaml", "r"), Loader=yaml.FullLoader)
print(config)

{'batch_size': 512, 'warm_up': 10, 'epochs': 100, 'load_model': 'None', 'eval_every_n_epochs': 1, 'save_every_n_epochs': 5, 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'weight_decay': '1e-5', 'gpu': 'cuda:0', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 512, 'drop_ratio': 0, 'pool': 'mean'}, 'aug': 'node', 'dataset': {'num_workers': 12, 'valid_size': 0.05, 'data_path': 'data/pubchem-10m-clean.txt'}, 'loss': {'temperature': 0.1, 'use_cosine_similarity': True}}


In [123]:
if config['aug'] == 'node':
    from dataset.dataset import MoleculeDatasetWrapper
elif config['aug'] == 'subgraph':
    from dataset.dataset_subgraph import MoleculeDatasetWrapper
elif config['aug'] == 'mix':
    from dataset.dataset_mix import MoleculeDatasetWrapper
else:
    raise ValueError('Not defined molecule augmentation!')

In [128]:
dataset = MoleculeDatasetWrapper(config['batch_size'], **config['dataset'])
molclr = MolCLR(dataset, config)
molclr.train()

Running on: cuda:0


AttributeError: 'MolCLR' object has no attribute '_load_pre_trained_weights'

In [115]:
import os
import csv
import math
import time
import random
import networkx as nx
import numpy as np
from copy import deepcopy


In [117]:
import torch
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader

In [118]:
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

from torch_scatter import scatter
from torch_geometric.data import Data, Dataset, DataLoader

In [119]:
import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem import AllChem

In [120]:
ATOM_LIST = list(range(1,119))
CHIRALITY_LIST = [
    Chem.rdchem.ChiralType.CHI_UNSPECIFIED,
    Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW,
    Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW,
    Chem.rdchem.ChiralType.CHI_OTHER
]
BOND_LIST = [
    BT.SINGLE, 
    BT.DOUBLE, 
    BT.TRIPLE, 
    BT.AROMATIC
]
BONDDIR_LIST = [
    Chem.rdchem.BondDir.NONE,
    Chem.rdchem.BondDir.ENDUPRIGHT,
    Chem.rdchem.BondDir.ENDDOWNRIGHT
]

In [121]:
class MoleculeDatasetWrapper(object):
    def __init__(self, batch_size, num_workers, valid_size, data_path):
        super(object, self).__init__()
        self.data_path = data_path
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.valid_size = valid_size

    def get_data_loaders(self):
        train_dataset = MoleculeDataset(data_path=self.data_path)
        train_loader, valid_loader = self.get_train_validation_data_loaders(train_dataset)
        return train_loader, valid_loader

    def get_train_validation_data_loaders(self, train_dataset):
        # obtain training indices that will be used for validation
        num_train = len(train_dataset)
        indices = list(range(num_train))
        np.random.shuffle(indices)

        split = int(np.floor(self.valid_size * num_train))
        train_idx, valid_idx = indices[split:], indices[:split]

        # define samplers for obtaining training and validation batches
        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, sampler=train_sampler,
                                  num_workers=self.num_workers, drop_last=True)

        valid_loader = DataLoader(train_dataset, batch_size=self.batch_size, sampler=valid_sampler,
                                  num_workers=self.num_workers, drop_last=True)

        return train_loader, valid_loader

In [1]:
pwd

'/tf/gpu/project/MolCLR-master'

In [ ]:
!python molclr.py

Please install apex for mixed precision training from: https://github.com/NVIDIA/apex
{'batch_size': 512, 'warm_up': 10, 'epochs': 100, 'load_model': 'None', 'eval_every_n_epochs': 1, 'save_every_n_epochs': 5, 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'weight_decay': '1e-5', 'gpu': 'cuda:0', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 512, 'drop_ratio': 0, 'pool': 'mean'}, 'aug': 'node', 'dataset': {'num_workers': 12, 'valid_size': 0.05, 'data_path': 'data/pubchem-10m-clean.txt'}, 'loss': {'temperature': 0.1, 'use_cosine_similarity': True}}
Running on: cuda:0
Pre-trained weights not found. Training from scratch.
GINet(
  (x_embedding1): Embedding(119, 300)
  (x_embedding2): Embedding(3, 300)
  (gnns): ModuleList(
    (0): GINEConv(
      (mlp): Sequential(
        (0): Linear(in_features=300, out_features=600, bias=True)
        (1): ReLU()
        (2): Linear(in_features=600, out_features=300, bias=True)
      )
      (edge_emb

0 10100 0.7314008474349976
0 10150 0.7496617436408997
0 10200 0.7093992829322815
0 10250 0.7071371078491211
0 10300 0.6820423603057861
0 10350 0.7570251822471619
0 10400 0.754153847694397
0 10450 0.7468463182449341
0 10500 0.7811804413795471
0 10550 0.7317599058151245
0 10600 0.721354603767395
0 10650 0.7256970405578613
0 10700 0.7795379161834717
0 10750 0.7771807909011841
0 10800 0.7231899499893188
0 10850 0.7157556414604187
0 10900 0.7792128324508667
0 10950 0.7632359266281128
0 11000 0.7880260348320007
0 11050 0.6952082514762878
0 11100 0.7016741037368774
0 11150 0.7679789066314697
0 11200 0.7310846447944641
0 11250 0.7048199772834778
0 11300 0.7066479325294495
0 11350 0.7199466228485107
0 11400 0.7396036982536316
0 11450 0.7134936451911926
0 11500 0.7077022790908813
0 11550 0.7470433115959167
0 11600 0.7302815914154053
0 11650 0.7290737628936768
0 11700 0.7388911247253418
0 11750 0.6923249959945679
0 11800 0.6568203568458557
0 11850 0.6392096877098083
0 11900 0.7274723649024963
0 1

1 7046 0.6482901573181152
1 7096 0.6115081906318665
1 7146 0.6148406863212585
1 7196 0.6202570796012878
1 7246 0.5999579429626465
1 7296 0.6063247919082642
1 7346 0.671029269695282
1 7396 0.660891592502594
1 7446 0.6612511873245239
1 7496 0.6039198040962219
1 7546 0.6285685300827026
1 7596 0.6376326084136963
1 7646 0.6441118121147156
1 7696 0.660845160484314
1 7746 0.6185458302497864
1 7796 0.6493748426437378
1 7846 0.6582935452461243
1 7896 0.6105042099952698
1 7946 0.6349340081214905
1 7996 0.5991353988647461
1 8046 0.6230016350746155
1 8096 0.6824066638946533
1 8146 0.6468411684036255
1 8196 0.6619328260421753
1 8246 0.6315435171127319
1 8296 0.6300155520439148
1 8346 0.6718010902404785
1 8396 0.6328220963478088
1 8446 0.6067482829093933
1 8496 0.602120041847229
1 8546 0.6362413167953491
1 8596 0.6546644568443298
1 8646 0.7100610136985779
1 8696 0.5923111438751221
1 8746 0.6351702213287354
1 8796 0.6085116267204285
1 8846 0.5868363976478577
1 8896 0.5936869978904724
1 8946 0.6197149

2 3992 0.5881063342094421
2 4042 0.5524306893348694
2 4092 0.5566606521606445
2 4142 0.5290767550468445
2 4192 0.6073513627052307
2 4242 0.5254282355308533
2 4292 0.6034347414970398
2 4342 0.5697680115699768
2 4392 0.565314769744873
2 4442 0.5866988897323608
2 4492 0.5872173309326172
2 4542 0.5784730315208435
2 4592 0.539391040802002
2 4642 0.5432063937187195
2 4692 0.5469750165939331
2 4742 0.5749425292015076
2 4792 0.6163029670715332
2 4842 0.6090794801712036
2 4892 0.562936544418335
2 4942 0.5956212282180786
2 4992 0.6503197550773621
2 5042 0.5863107442855835
2 5092 0.5646772980690002
2 5142 0.6026695966720581
2 5192 0.553615927696228
2 5242 0.5598241090774536
2 5292 0.5919287800788879
2 5342 0.5521770119667053
2 5392 0.5903205275535583
2 5442 0.553011417388916
2 5492 0.5738405585289001
2 5542 0.6133208274841309
2 5592 0.5426809191703796
2 5642 0.5326550006866455
2 5692 0.5357821583747864
2 5742 0.49171122908592224
2 5792 0.5714027285575867
2 5842 0.5385212898254395
2 5892 0.5755786

3 938 0.5533930659294128
3 988 0.5541414618492126
3 1038 0.5540438294410706
3 1088 0.5062717795372009
3 1138 0.5282575488090515
3 1188 0.606834352016449
3 1238 0.550322949886322
3 1288 0.566547691822052
3 1338 0.5306644439697266
3 1388 0.5185941457748413
3 1438 0.5228261351585388
3 1488 0.5621181726455688
3 1538 0.6447569131851196
3 1588 0.5127366185188293
3 1638 0.5163987278938293
3 1688 0.5419908165931702
3 1738 0.5164138674736023
3 1788 0.5600470304489136
3 1838 0.5723443627357483
3 1888 0.48897451162338257
3 1938 0.5258831977844238
3 1988 0.5354898571968079
3 2038 0.5123716592788696
3 2088 0.566475510597229
3 2138 0.5689278841018677
3 2188 0.5199266672134399
3 2238 0.5480701923370361
3 2288 0.608303427696228
3 2338 0.5368753671646118
3 2388 0.5188682675361633
3 2438 0.5297253727912903
3 2488 0.5518653988838196
3 2538 0.5760819315910339
3 2588 0.5382855534553528
3 2638 0.5353309512138367
3 2688 0.5762307047843933
3 2738 0.5230857729911804
3 2788 0.5214598774909973
3 2838 0.564092576

...

27 5042 0.4829961061477661
27 5092 0.486549973487854
27 5142 0.48999258875846863
27 5192 0.47103601694107056
27 5242 0.5190619826316833
27 5292 0.46346530318260193
27 5342 0.47207608819007874
27 5392 0.5004280805587769
27 5442 0.4750356674194336
27 5492 0.47086626291275024
27 5542 0.4862935543060303
27 5592 0.46973124146461487
27 5642 0.47131434082984924
27 5692 0.539752721786499
27 5742 0.5025217533111572
27 5792 0.4939826428890228
27 5842 0.4672979414463043
27 5892 0.47274109721183777
27 5942 0.49395138025283813
27 5992 0.44740480184555054
27 6042 0.4785345792770386
27 6092 0.5398831367492676
27 6142 0.5451477766036987
27 6192 0.4704180359840393
27 6242 0.4929245114326477
27 6292 0.5432571172714233
27 6342 0.5116862654685974
27 6392 0.45992374420166016
27 6442 0.42088785767555237
27 6492 0.49409398436546326
27 6542 0.4817258417606354
27 6592 0.4735839366912842
27 6642 0.451680451631546
27 6692 0.5081220269203186
27 6742 0.49129152297973633
27 6792 0.44931304454803467
27 6842 0.490872

28 1188 0.45120173692703247
28 1238 0.48597946763038635
28 1288 0.50736403465271
28 1338 0.5108840465545654
28 1388 0.42139580845832825
28 1438 0.517117977142334
28 1488 0.5157127380371094
28 1538 0.4508103132247925
28 1588 0.4791393578052521
28 1638 0.44164079427719116
28 1688 0.4750719368457794
28 1738 0.5065460205078125
28 1788 0.5025299787521362
28 1838 0.46607664227485657
28 1888 0.43596503138542175
28 1938 0.42092427611351013
28 1988 0.4345167875289917
28 2038 0.49545517563819885
28 2088 0.4736359715461731
28 2138 0.46683943271636963
28 2188 0.46222740411758423
28 2238 0.43457818031311035
28 2288 0.42548200488090515
28 2338 0.4369969666004181
28 2388 0.5189586877822876
28 2438 0.4692704379558563
28 2488 0.4614260196685791
28 2538 0.488613098859787
28 2588 0.4846017360687256
28 2638 0.5066619515419006
28 2688 0.47279825806617737
28 2738 0.4651818871498108
28 2788 0.49386465549468994
28 2838 0.5000972151756287
28 2888 0.485166996717453
28 2938 0.4651449918746948
28 2988 0.535212397

28 16038 0.48784139752388
28 16088 0.4663395881652832
28 16138 0.4564306437969208
28 16188 0.5212814807891846
28 16238 0.4895111918449402
28 16288 0.4993729293346405
28 16338 0.49561360478401184
28 16388 0.48144450783729553
28 16438 0.5164838433265686
28 16488 0.5264375805854797
28 16538 0.5001275539398193
28 16588 0.4383331537246704
28 16638 0.5038129687309265
28 16688 0.47673046588897705
28 16738 0.5169887542724609
28 16788 0.44894975423812866
28 16838 0.47335827350616455
28 16888 0.4512312710285187
28 16938 0.5041195750236511
28 16988 0.4915601909160614
28 17038 0.47553279995918274
28 17088 0.49416646361351013
28 17138 0.4691377282142639
28 17188 0.4909367859363556
28 17238 0.46646741032600403
28 17288 0.4776811897754669
28 17338 0.46843042969703674
28 17388 0.4527488648891449
28 17438 0.48132026195526123
28 17488 0.4352766275405884
28 17538 0.48194292187690735
28 17588 0.497187077999115
28 17638 0.4770687520503998
28 17688 0.44863083958625793
28 17738 0.5333036780357361
28 17788 0.

29 12334 0.47425442934036255
29 12384 0.5582625865936279
29 12434 0.4964892566204071
29 12484 0.42118486762046814
29 12534 0.5594404339790344
29 12584 0.4419999122619629
29 12634 0.4813944399356842
29 12684 0.4915262758731842
29 12734 0.49029839038848877
29 12784 0.43805399537086487
29 12834 0.5242190361022949
29 12884 0.42531946301460266
29 12934 0.4791278541088104
29 12984 0.49750351905822754
29 13034 0.4712631106376648
29 13084 0.44668444991111755
29 13134 0.4788118898868561
29 13184 0.4909212589263916
29 13234 0.5041050314903259
29 13284 0.49521172046661377
29 13334 0.4200926721096039
29 13384 0.515539824962616
29 13434 0.4924415051937103
29 13484 0.430860310792923
29 13534 0.4566105902194977
29 13584 0.5016940832138062
29 13634 0.5042527318000793
29 13684 0.44609159231185913
29 13734 0.4478096067905426
29 13784 0.44623124599456787
29 13834 0.5233455896377563
29 13884 0.4489506483078003
29 13934 0.43048393726348877
29 13984 0.4729965031147003
29 14034 0.45756596326828003
29 14084 0

30 8530 0.5218443274497986
30 8580 0.44783449172973633
30 8630 0.45367878675460815
30 8680 0.4553329646587372
30 8730 0.4739667475223541
30 8780 0.44693809747695923
30 8830 0.5307129621505737
30 8880 0.5001907348632812
30 8930 0.45953625440597534
30 8980 0.4070252776145935
30 9030 0.4855502247810364
30 9080 0.4465331435203552
30 9130 0.4795686900615692
30 9180 0.5202193856239319
30 9230 0.48034411668777466
30 9280 0.4942249059677124
30 9330 0.48407161235809326
30 9380 0.49517950415611267
30 9430 0.43762052059173584
30 9480 0.4946248531341553
30 9530 0.48015886545181274
30 9580 0.47430044412612915
30 9630 0.4638747274875641
30 9680 0.47189146280288696
30 9730 0.4884042739868164
30 9780 0.45830389857292175
30 9830 0.49517881870269775
30 9880 0.506488561630249
30 9930 0.4876508116722107
30 9980 0.501973032951355
30 10030 0.47037869691848755
30 10080 0.4722946584224701
30 10130 0.4730171859264374
30 10180 0.47943615913391113
30 10230 0.46466609835624695
30 10280 0.4829545021057129
30 10330

In [1]:
import os
# datasets = ["bace",  "bbbp", "tox21", "toxcast", "sider",]
seeds = [777, 778, 779, 780, 781]

for seed in seeds:
    cmd = f"\
    python finetune.py \
    --seed {seed} "

    os.system(cmd)

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


{'batch_size': 32, 'epochs': 200, 'eval_every_n_epochs': 1, 'fine_tune_from': 'pretrained_gin', 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'init_base_lr': 0.0001, 'weight_decay': '1e-6', 'gpu': 'cuda:1', 'task_name': 'BBBP', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 300, 'drop_ratio': 0.3, 'pool': 'mean'}, 'dataset': {'num_workers': 4, 'valid_size': 0.1, 'test_size': 0.1, 'splitting': 'scaffold', 'seed': 777, 'task': 'classification', 'data_path': 'data/bbbp/BBBP.csv'}}
Running on: cuda:1
2038
2038
About to generate scaffolds
Generating scaffold 0/2038
Generating scaffold 1000/2038
Generating scaffold 2000/2038
About to sort in scaffold sets
pred_head.0.weight True
pred_head.0.bias True
pred_head.2.weight True
pred_head.2.bias True
pred_head.4.weight True
pred_head.4.bias True
0 0 0.5263356566429138
0 50 0.2996859550476074
Validation loss: 0.7322044384245779 ROC AUC: 0.8986801242236024
1 49 0.5510687828063965
Validation loss: 0

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


{'batch_size': 32, 'epochs': 200, 'eval_every_n_epochs': 1, 'fine_tune_from': 'pretrained_gin', 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'init_base_lr': 0.0001, 'weight_decay': '1e-6', 'gpu': 'cuda:1', 'task_name': 'BBBP', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 300, 'drop_ratio': 0.3, 'pool': 'mean'}, 'dataset': {'num_workers': 4, 'valid_size': 0.1, 'test_size': 0.1, 'splitting': 'scaffold', 'seed': 778, 'task': 'classification', 'data_path': 'data/bbbp/BBBP.csv'}}
Running on: cuda:1
2038
2038
About to generate scaffolds
Generating scaffold 0/2038
Generating scaffold 1000/2038
Generating scaffold 2000/2038
About to sort in scaffold sets
pred_head.0.weight True
pred_head.0.bias True
pred_head.2.weight True
pred_head.2.bias True
pred_head.4.weight True
pred_head.4.bias True
0 0 0.6113183498382568
0 50 0.368680864572525
Validation loss: 0.6960037040944193 ROC AUC: 0.9023680124223602
1 49 0.364168256521225
Validation loss: 0.4

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


{'batch_size': 32, 'epochs': 200, 'eval_every_n_epochs': 1, 'fine_tune_from': 'pretrained_gin', 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'init_base_lr': 0.0001, 'weight_decay': '1e-6', 'gpu': 'cuda:1', 'task_name': 'BBBP', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 300, 'drop_ratio': 0.3, 'pool': 'mean'}, 'dataset': {'num_workers': 4, 'valid_size': 0.1, 'test_size': 0.1, 'splitting': 'scaffold', 'seed': 779, 'task': 'classification', 'data_path': 'data/bbbp/BBBP.csv'}}
Running on: cuda:1
2038
2038
About to generate scaffolds
Generating scaffold 0/2038
Generating scaffold 1000/2038
Generating scaffold 2000/2038
About to sort in scaffold sets
pred_head.0.weight True
pred_head.0.bias True
pred_head.2.weight True
pred_head.2.bias True
pred_head.4.weight True
pred_head.4.bias True
0 0 0.5171658396720886
0 50 0.3072924017906189
Validation loss: 0.692314649329466 ROC AUC: 0.8845108695652174
1 49 0.5107566118240356
Validation loss: 0.

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


{'batch_size': 32, 'epochs': 200, 'eval_every_n_epochs': 1, 'fine_tune_from': 'pretrained_gin', 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'init_base_lr': 0.0001, 'weight_decay': '1e-6', 'gpu': 'cuda:1', 'task_name': 'BBBP', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 300, 'drop_ratio': 0.3, 'pool': 'mean'}, 'dataset': {'num_workers': 4, 'valid_size': 0.1, 'test_size': 0.1, 'splitting': 'scaffold', 'seed': 780, 'task': 'classification', 'data_path': 'data/bbbp/BBBP.csv'}}
Running on: cuda:1
2038
2038
About to generate scaffolds
Generating scaffold 0/2038
Generating scaffold 1000/2038
Generating scaffold 2000/2038
About to sort in scaffold sets
pred_head.0.weight True
pred_head.0.bias True
pred_head.2.weight True
pred_head.2.bias True
pred_head.4.weight True
pred_head.4.bias True
0 0 0.6661053895950317
0 50 0.40758267045021057
Validation loss: 0.5289506725236481 ROC AUC: 0.8150232919254659
1 49 0.13983948528766632
Validation loss:

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


{'batch_size': 32, 'epochs': 200, 'eval_every_n_epochs': 1, 'fine_tune_from': 'pretrained_gin', 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'init_base_lr': 0.0001, 'weight_decay': '1e-6', 'gpu': 'cuda:1', 'task_name': 'BBBP', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 300, 'drop_ratio': 0.3, 'pool': 'mean'}, 'dataset': {'num_workers': 4, 'valid_size': 0.1, 'test_size': 0.1, 'splitting': 'scaffold', 'seed': 781, 'task': 'classification', 'data_path': 'data/bbbp/BBBP.csv'}}
Running on: cuda:1
2038
2038
About to generate scaffolds
Generating scaffold 0/2038
Generating scaffold 1000/2038
Generating scaffold 2000/2038
About to sort in scaffold sets
pred_head.0.weight True
pred_head.0.bias True
pred_head.2.weight True
pred_head.2.bias True
pred_head.4.weight True
pred_head.4.bias True
0 0 0.8174243569374084
0 50 0.5125200748443604
Validation loss: 0.8626600924660178 ROC AUC: 0.8531638198757765
1 49 0.32613664865493774
Validation loss: 